In [1]:

import pandas as pd

Scraping Wikipedia page in order to obtain the data that is in the table of postal codes of Canada and read it into a pandas dataframe:

In [ ]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

The dataframe consists of three columns: PostalCode, Borough, and Neighborhood.

In [ ]:
df.head()

Ignoring cells with a borough that is Not assigned.

In [ ]:
print('Initial number of rows:', df.shape)
df = df[df['Borough']!='Not assigned']
print('Number of rows after ignoring unassigned boroughs:', df.shape)

The number of rows is equal to the number of unique postal codes (multiple neighbourhoods with the same postal code have already been merged).

In [ ]:
df['Postal Code'].nunique()

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough --> This constraint is already respected, there are no unassigned neighbourhoods.

In [ ]:
df[df['Neighbourhood']=='Not assigned']

In [ ]:
print('number of rows in the dataframe:', df.shape[0])